In [1]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [2]:
# 1. Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    movie = dict(movie) # Convert list item to a dict
    alt_titles = {} # Initialize a dictionary to hold alt titles
    
    # Column headers that could hold alt titles
    alt_keys = ['Also known as', 'Arabic', 'Cantonese', 'Chinese', 'French', 'Hangul', 'Hebrew', 'Hepburn', 'Japanese', 
                'Literally', 'Mandarin', 'McCune–Reischauer', 'Original title', 'Polish', 'Revised Romanization', 
                'Romanized', 'Russian', 'Simplified', 'Traditional', 'Yiddish']
    
    # Check each key for data
    for key in alt_keys:

        # If data exists in current key, add it to alt titles dict and remove it from movie dict
        if key in movie: 
            alt_titles[key] = movie[key]
            movie.pop(key)

    # If current movie contains alt titles, add them under alt_titles key to original dict
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles

    # Create a function that will combine like-columns based on the tuple "names"
    def change_column_name(names):
        if names[0] in movie:
            movie[names[1]] = movie.pop(names[0])

    # A list of tuples that will be passed to the change_column_name function. First entry is old name, second is new name.
    columns_to_change = [
        ('Adaptation by', 'Writer(s)'),
        ('Screen story by', 'Writer(s)'),
        ('Screenplay by', 'Writer(s)'),
        ('Story by', 'Writer(s)'),
        ('Written by', 'Writer(s)'),
        ('Adaptation by', 'Writer(s)'),
        ('Country of origin', 'Country'),
        ('Directed by','Director(s)'),
        ('Director','Director(s)'),
        ('Created by', 'Creator(s)'),
        ('Distributed by','Distributor(s)'),
        ('Distributor','Distributor(s)'),
        ('Edited by', 'Editor(s)'),
        ('Length', 'Running time'),
        ('Music by', 'Composer(s)'),
        ('Original release', 'Release date'),
        ('Produced by', 'Producer(s)'),
        ('Producer', 'Producer(s)'),
        ('Production company(s)', 'Production compan(ies)'),
        ('Productioncompanies ', 'Production compan(ies)'),
        ('Productioncompany ', 'Production compan(ies)'),
        ('Released', 'Release date'),
        ('Theme music composer', 'Composer(s)')
    ]

    [change_column_name(names) for names in columns_to_change]

    return movie

In [3]:
# 2 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def movies_ETL():
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_metadata = pd.read_csv('Resources/movies_metadata.csv', low_memory=False)
    ratings = pd.read_csv('Resources/ratings.csv')

    # Open and read the Wikipedia data JSON file.
    with open('Resources/wikipedia-movies.json', mode='r') as file:
        wiki_movies = json.load(file)
    
    # 3. Write a list comprehension to filter out TV shows.
    wiki_movies = [movie for movie in wiki_movies if 'No. of episodes' not in movie]

    # 4. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    clean_wiki_movies = [clean_movie(movie) for movie in wiki_movies]

    # 5. Read in the cleaned movies list from Step 4 as a DataFrame.
    wiki_movies_df = pd.DataFrame(clean_wiki_movies)

    # 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try:
        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
    except Exception as e:
        print(e)

    #  7. Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    columns_to_keep = [col for col in wiki_movies_df.columns if wiki_movies_df[col].isnull().sum() < len(wiki_movies_df)]
    wiki_movies_df = wiki_movies_df[columns_to_keep]

    # 8. Create a variable that will hold the non-null values from the “Box office” column.
    box_office = wiki_movies_df['Box office'].dropna()
    
    # 9. Convert the box office data created in Step 8 to string values using the lambda and join functions.
    box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)

    # 10. Write a regular expression to match the six elements of "form_one" of the box office data.
    form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'

    # 11. Write a regular expression to match the three elements of "form_two" of the box office data.
    form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'

    # 12. Add the parse_dollars function.
    def parse_dollars(s):
    # if s is not a string, return NaN
        if type(s) != str:
            return np.nan

        # if input is of the form $###.# million
        if re.match(r"\$\s*\d+\.?\d*\s*milli?on", s, flags=re.IGNORECASE):
            # remove the $ and million
            s = re.sub(r'\$|\s|[a-zA-Z]','', s)
            # convert to float and multiply my a million
            s = float(s) * 10 ** 6
            # return value
            return s
        # if input is of the form $###.# billion
        elif re.match(r"\$\s*\d+\.?\d*\s*billi?on", s, flags=re.IGNORECASE):
            # remove the $ and billion
            s = re.sub(r'\$|\s|[a-zA-Z]', '', s)
            # convert to float and multiply my a billion
            s = float(s) * 10 ** 9
            # return value
            return s
        # if input is of the form $###,###,###
        elif re.match(form_two, s):
            # Remove the $ and ,
            s = re.sub(r'\$|,', '', s)
            # convert to float
            s = float(s)
            # return value
            return s
        # otherwise, return NaN
        else:
            return np.nan
        
    # 13. Clean the box office column in the wiki_movies_df DataFrame.
    wiki_movies_df['Box office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    
    # 14. Clean the budget column in the wiki_movies_df DataFrame.
    wiki_movies_df['Budget'] = wiki_movies_df.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

    # 15. Clean the release date column in the wiki_movies_df DataFrame.
    

    # 16. Clean the running time column in the wiki_movies_df DataFrame.
    
    # Return three variables. The first is the wiki_movies_df DataFrame
    
    return wiki_movies_df, kaggle_metadata, ratings 

In [4]:
# 17. Create the path to your file directory and variables for the three files.
file_dir = 
# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia.movies.json'
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [5]:
# 18. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = extract_transform_load()

In [6]:
# 19. Set the wiki_movies_df equal to the wiki_file variable. 
wiki_movies_df = wiki_file

In [7]:
# 20. Check that the wiki_movies_df DataFrame looks like this. 
wiki_movies_df.head()

In [8]:
# 21. Check that wiki_movies_df DataFrame columns are correct. 
wiki_movies_df.columns.to_list()